In [1]:
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import ensemble
from sklearn import multioutput
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler
import joblib
import config as cnf
import Utils as utils

c:\Users\HAMZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


## Warnings

In [2]:
# For output settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 700)

# For warnings
warnings.filterwarnings("ignore")

## Load Dataset

In [3]:
train_df = pd.read_parquet(cnf.trainpath)
test = pd.read_parquet(cnf.testpath)

## Target

In [4]:
def do_Target_spareted(dataframe):
    new_spareted_cabin = dataframe["target"].str.split(pat = ",", expand = True)
    
    dataframe.drop("target", axis=1, inplace=True)
    
    new_spareted_cabin.rename(columns={0 : 'first_menu',
                                       1 : 'second_menu',
                                       2 : 'third_menu'}, inplace=True)
    
    return pd.concat([dataframe, new_spareted_cabin], axis=1)

In [5]:
train_df = do_Target_spareted(train_df)
def binarize_column(column):
    lb = LabelBinarizer()
    transformed_data = lb.fit_transform(column)
    if column.name == "second_menu":
        transformed_data = [np.insert(row, 4, 0) for row in transformed_data]
    elif column.name == "third_menu":
        transformed_data = [np.insert(row, 2, 0) for row in transformed_data]
    return pd.Series([row.tolist() for row in transformed_data])

train_df["first_menu"] = binarize_column(train_df["first_menu"])
train_df["second_menu"] = binarize_column(train_df["second_menu"])
train_df["third_menu"] = binarize_column(train_df["third_menu"])

train_df['target'] = train_df[['first_menu', 'second_menu', 'third_menu']].apply(
    lambda row: [1 if any(x) else 0 for x in zip(*row)], axis=1
)

train_df.drop(["first_menu","second_menu","third_menu"], inplace=True, axis=1)

In [6]:
train_df = train_df.drop_duplicates(subset='id', keep='first')
test = test.drop_duplicates(subset='id', keep='first')

In [7]:
train_df.drop(utils.grab_outliers(train_df,'n_seconds_3',index=True),axis=0,inplace=True)
# train_df.drop(utils.grab_outliers(train_df,'n_seconds_2',index=True),axis=0,inplace=True) kotu etkiledi

                              id  month  n_seconds_1  n_seconds_2  n_seconds_3       carrier devicebrand  feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  feature_8  feature_9  feature_10  feature_11  feature_12  feature_13  feature_14  feature_15  feature_16  feature_17  feature_18  feature_19  feature_20  feature_21  feature_22  feature_23  feature_24  feature_25  feature_26  feature_27  feature_28  feature_29  feature_30  feature_31  feature_32  feature_33  feature_34  feature_35  feature_36  feature_37  feature_38  feature_39  feature_40  feature_41  feature_42  feature_43  feature_44  feature_45  feature_46  feature_47  feature_48  feature_49  \
5      08773130c89b00852a36a6c5b     10     3011.760     1929.067     1183.204      TURKCELL     samsung  -1.703797   1.908199   0.647938  -2.648655   4.140598   2.995900  -0.534020   0.442913   0.358865  -2.102661   -0.104798   -0.847690    2.611740   -0.833706    1.885863   -2.304482   -2.252935   -

In [8]:
len(utils.grab_outliers(train_df,'n_seconds_1',index=True))

                              id  month  n_seconds_1  n_seconds_2  n_seconds_3       carrier devicebrand  feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  feature_8  feature_9  feature_10  feature_11  feature_12  feature_13  feature_14  feature_15  feature_16  feature_17  feature_18  feature_19  feature_20  feature_21  feature_22  feature_23  feature_24  feature_25  feature_26  feature_27  feature_28  feature_29  feature_30  feature_31  feature_32  feature_33  feature_34  feature_35  feature_36  feature_37  feature_38  feature_39  feature_40  feature_41  feature_42  feature_43  feature_44  feature_45  feature_46  feature_47  feature_48  feature_49  \
15131  ff45f2c3fe6a68d18c42c94e1     11     7437.251      328.532      262.480  TURK TELEKOM     samsung  -3.400307   2.738383  -0.960845   0.049726   3.887482   2.453614  -0.538428   0.067862   0.907018  -4.547372   -1.195070   -1.009250    1.699859   -2.335222    2.985204    0.004496   -2.861916   -

34

In [9]:
len(utils.grab_outliers(test,'n_seconds_1',index=True))

                          id  month  n_seconds_1  n_seconds_2  n_seconds_3       carrier devicebrand  feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  feature_8  feature_9  feature_10  feature_11  feature_12  feature_13  feature_14  feature_15  feature_16  feature_17  feature_18  feature_19  feature_20  feature_21  feature_22  feature_23  feature_24  feature_25  feature_26  feature_27  feature_28  feature_29  feature_30  feature_31  feature_32  feature_33  feature_34  feature_35  feature_36  feature_37  feature_38  feature_39  feature_40  feature_41  feature_42  feature_43  feature_44  feature_45  feature_46  feature_47  feature_48  feature_49
0  2e6105f5911256f4f6c4813ed      1     6893.544      246.854      242.636   VODAFONE TR     samsung  -1.723524   3.216489  -1.138474   2.026997   2.241670   1.796100  -0.212805   0.447929   3.465160  -4.219648   -0.931751    3.633603   -0.555067   -2.298111    0.511194   -0.383306   -2.593233   -2.447223   

5

In [10]:
X = train_df.drop("target", axis=1)
y = train_df["target"]

numpy_dizi = y.to_numpy()
numpy_dizi = [np.array(row) for row in numpy_dizi]

In [11]:
print(train_df['devicebrand'].unique())
a = train_df['devicebrand'].unique()

['Apple' 'samsung' 'HUAWEI' 'Redmi' 'POCO' 'OPPO' 'Sony' 'vivo' 'asus'
 'gm' 'Alcatel' 'xiaomi' 'reeder' 'HONOR' 'OMIX' 'Lenovo' 'realme' 'Meizu'
 'TCL' 'GM' 'Vestel' 'TECNO' 'lge' 'Casper' 'CASPER' 'motorola' 'OnePlus'
 'htc' 'Turk_Telekom' 'Nokia' 'Ulefone' 'google' 'HIKING' 'Reeder'
 'Trident' 'ZTE' 'generalmobile' 'blackberry' 'Elephone' 'Hytera'
 'Blackview' 'DOOGEE' 'SuperD' 'OUKITEL' 'Infinix' 'Vodafone' 'Realme'
 'UMIDIGI' 'nubia' 'Huawei' 'meizu' 'Nothing' 'DIJITSU' 'HTC' 'HiKING'
 'TURKCELL' 'Fairphone' 'iBRIT' 'KAAN' 'Cat' 'HOMETECH' 'WIKO' 'Gigaset'
 'blackshark']


In [12]:
def dataprep1(df):
    for i in ['gm','GM','generalmobile']:
        df['devicebrand'] = df['devicebrand'].str.replace(i, 'GENERAL_MOBILE', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('Reeder', 'reeder', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('Casper', 'CASPER', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('Realme', 'realme', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('htc', 'HTC', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('HikING', 'HIKING', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('Huawei', 'HUAWEI', regex=True)
    df['devicebrand'] = df['devicebrand'].str.replace('Meizu', 'meizu', regex=True)
    
    
    df['carrier'] = df['carrier'].apply(lambda x: 'VODAFONE' if 'VODAFONE' in x else x)
    df['carrier'] = df['carrier'].apply(lambda x: 'VODAFONE' if 'VF' in x else x)
    df['carrier'] = df['carrier'].apply(lambda x: 'TURKCELL' if 'LIFECELL' in x else x)
    df['carrier'] = df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURK TELEKO' in x else x)
    df['carrier'] = df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TÜRK TELEKO' in x else x)
    df['carrier'] = df['carrier'].apply(lambda x: 'TURK_TELEKOM' if 'TURKTELEKOM' in x else x)
    df['carrier'] = df['carrier'].str.replace('^KCELL ', 'AKCELL', regex=True)
    df['carrier'] = df['carrier'].str.replace('^中国', 'CHINATEL', regex=True)
    df['carrier'] = df['carrier'].str.replace('ドコモ', 'JAPENTEL', regex=True)  
    df['carrier'] = df['carrier'].str.replace('^Z 4.5G+', 'ZAIN', regex=True)  


    yurtdisi = ['ALMADAR','AIRTEL','AZERCELL','BAKCELL','O2','BEE','A1 ','3_AT','IRANCELL',
                'AYYILDIZ','BH','NL','ORANGE','MOLDCELL','ZAIN','YETTEL','VERIZONE','TELEKOM',
                'TELENOR','TELE2','TELIA','MAGTI','STC','BOUYGUES','HORMUUD','JIO','LIDL','KSA',
                'FREEDOM','BUDGET','XFINITY','CHINA_TELECOM','MTN','1&1','BASE','CLARO','GEOCELL',
                'MEGAFONE','GSMOBILE','ETISALAT','TIM','MAXIS','PROXIMUS', 'SUNRISE', 'WINDTRE', 
                'VODACOM', 'LYCAMOBILE','LIBYANA','TIGO', 'ASIACELL', 'SFR','CUBACELL','AKCELL',
                'SALT','T-MOBILE', 'CHINATEL', 'JAPENTEL','ROBI','AWCC','KYIVSTAR','GLOBE','TDC',
                'DIGICEL','DIGITEL','DIGITEC','VOLNA','HANDYVERTRAG',] 
    sirket = ['TURKCELL','BIMCELL','AVEA','VODAFONE','TURK_TELEKOM','TEKNOSA','PTTCELL','KKTCELL','PRIMETEL']
    
    birlesim = yurtdisi+sirket
    
    for anahtarKelime in birlesim:
        df['carrier'] = df['carrier'].apply(lambda x: anahtarKelime if anahtarKelime in x else x)
    
    for anahtarKelime in ['HAYAT','FENER','TRABZON','61','UNKNOWN',' ','nknown']:  
        df['carrier'] = df['carrier'].apply(lambda x: 'UNKNOWN' if anahtarKelime in x else x)
    
    unknown = [row for row in df['carrier'].unique() if row not in birlesim]
    df['carrier'] = df['carrier'].apply(lambda x: 'UNKNOWN' if x in unknown else x if x not in unknown else x)
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
    # df['is_ios'] = df['devicebrand'].apply(lambda x: 1 if x.lower() == 'apple' else 0)
    df[['carrier', 'devicebrand']] = ordinal_encoder.fit_transform(df[['carrier', 'devicebrand']])
    # df['total_nseconds'] = df['n_seconds_1'] + df['n_seconds_2'] + df['n_seconds_3']

    return df

## TRAIN

In [13]:
train_df = dataprep1(train_df)

In [14]:
X = train_df.drop("target", axis=1)
y = train_df["target"]
X = X.drop(["id"], axis=1)

## TEST

In [15]:
test = dataprep1(test)

In [16]:
test.drop('id',axis=1,inplace=True)

In [17]:
# reg = multioutput.MultiOutputRegressor(ensemble.GradientBoostingRegressor())
# reg.fit(X, numpy_dizi)
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

lgb_model = lgb.LGBMRegressor(learning_rate=0.05, n_estimators=500, max_depth=7, num_leaves=35)
model = MultiOutputRegressor(lgb_model)

model = model.fit(X, numpy_dizi)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011800 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13243
[LightGBM] [Info] Number of data points in the train set: 80453, number of used features: 56
[LightGBM] [Info] Start training from score 0.239320
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13243
[LightGBM] [Info] Number of data points in the train set: 80453, number of used features: 56
[LightGBM] [Info] Start training from score 0.797708
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011496 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13243
[LightGBM] [Info] Number of data points in the train set: 80453, number of used features: 56
[LightGBM] [Inf

In [18]:
y_pred = model.predict(test)

In [19]:
def sample_sub(ypred):
    

    sample = pd.read_csv("csv_sample.csv")

    submission = pd.DataFrame({"id": sample["id"],
                                "target": ypred})
    # submission['target'] = submission['target'].apply(lambda x: ''.join(map(str, eval(x))))
                                
    submission.to_csv("22.csv", index=False)

In [20]:
for row in y_pred:
    list_with_index = [(value, index) for index, value in enumerate(row)]
    sorted_list = sorted(list_with_index, key=lambda x: x[0])
    three_values = [x[1] for x in sorted_list[-3:]]
    row[three_values]=1
    row[~np.isin(np.arange(len(row)), three_values)] = 0
y_pred = y_pred.astype(int)
y_pred = [row.tolist() for row in y_pred]
print(y_pred)

[[0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 1, 

In [21]:
sample_sub(pd.Series(y_pred))

In [22]:
df = pd.read_csv('22.csv')
df['target'] = df['target'].apply(lambda x: ''.join(map(str, eval(x))))

In [23]:
df.to_csv('022.csv',index=False)

In [24]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv("022.csv")
y_pred = pd.read_csv('0control.csv')
accuracy = accuracy_score(y_test["target"], y_pred["target"])

print("Accuracy:", accuracy)

Accuracy: 0.8844834797156002
